In [1]:
import keras
import tensorflow as tf
import torch
import numpy as np
from config import cfg

In [2]:

keras_model = keras.models.load_model('weights/prediction_weights.hdf5')
#weights = model.get_weights()

#tf_model = tf.keras.estimator.model_to_estimator(keras_model, model_dir = './model_estimator')
#tf_model._model_dir = './model_estimator/keras'

In [3]:
keras_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_imgs (InputLayer)     [(None, 480, 480, 3)]        0         []                            
                                                                                                  
 input_layouts (InputLayer)  [(None, 480, 480, 3)]        0         []                            
                                                                                                  
 concatenate_1 (Concatenate  (None, 480, 480, 6)          0         ['input_imgs[0][0]',          
 )                                                                   'input_layouts[0][0]']       
                                                                                                  
 conv1 (Conv2D)              (None, 237, 237, 64)         18880     ['concatenate_1[0][0]'] 

In [4]:
for layer in keras_model.layers:
    #if len(layer.weights) > 0:
        print(layer.name,":", len(layer.weights))

input_imgs : 0
input_layouts : 0
concatenate_1 : 0
conv1 : 2
bn_conv1 : 4
activation_1 : 0
max_pooling2d_1 : 0
res2a_branch2a : 2
bn2a_branch2a : 4
activation_2 : 0
res2a_branch2b : 2
bn2a_branch2b : 4
activation_3 : 0
res2a_branch2c : 2
res2a_branch1 : 2
bn2a_branch2c : 4
bn2a_branch1 : 4
add_1 : 0
activation_4 : 0
res3a_branch2a : 2
bn3a_branch2a : 4
activation_5 : 0
res3a_branch2b : 2
bn3a_branch2b : 4
activation_6 : 0
res3a_branch2c : 2
res3a_branch1 : 2
bn3a_branch2c : 4
bn3a_branch1 : 4
add_2 : 0
activation_7 : 0
res4a_branch2a : 2
bn4a_branch2a : 4
activation_8 : 0
res4a_branch2b : 2
bn4a_branch2b : 4
activation_9 : 0
res4a_branch2c : 2
res4a_branch1 : 2
input_rois : 0
bn4a_branch2c : 4
bn4a_branch1 : 4
reshape_2 : 0
add_3 : 0
repeat_vector_1 : 0
activation_10 : 0
permute_1 : 0
conv2d_3 : 2
reshape_3 : 0
conv2d_1 : 2
multiply_1 : 0
conv2d_2 : 2
global_max_pooling2d_1 : 0
reshape_1 : 0
dense_1 : 2
output_cens : 0
output_sizes : 2


In [5]:
print(keras_model.get_layer('dense_1').weights[0].shape)

print(keras_model.get_layer('res2a_branch2a').weights[0].name)
print(keras_model.get_layer('res2a_branch2a').weights[1].name)
print(keras_model.get_layer('bn_conv1').weights[2].name)
print(keras_model.get_layer('bn_conv1').weights[3].name)

(512, 225)
res2a_branch2a/kernel:0
res2a_branch2a/bias:0
bn_conv1/moving_mean:0
bn_conv1/moving_variance:0


In [6]:
from centerinference import CenterInferenceModel
from sizeinference import SizeInferenceModel

def get_weights(name, index, transpose = False):
    if transpose:
        return np.transpose(keras_model.get_layer(name).weights[index].numpy())
    return keras_model.get_layer(name).weights[index].numpy()

def set_conv_weights(conv, name):
    conv.weight.data = torch.from_numpy(np.transpose(get_weights(name, 0)))
    conv.bias.data = torch.from_numpy(get_weights(name, 1))

def set_dense_weights(dense, name):
    dense.weight.data = torch.from_numpy(np.transpose(get_weights(name, 0)))
    dense.bias.data = torch.from_numpy(get_weights(name, 1))

def set_bn_weights(bn, name):
    bn.weight.data = torch.from_numpy(np.transpose(get_weights(name, 0)))
    bn.bias.data = torch.from_numpy(get_weights(name, 1))
    bn.running_mean = torch.from_numpy(get_weights(name, 2))
    bn.running_var = torch.from_numpy(get_weights(name, 3))

def set_conv_block_weights(conv_block, stage):
    set_conv_weights(conv_block.conv1, f"res{stage}a_branch2a")
    set_conv_weights(conv_block.conv2, f"res{stage}a_branch2b")
    set_conv_weights(conv_block.conv3, f"res{stage}a_branch2c")

    set_bn_weights(conv_block.bn1, f"bn{stage}a_branch2a")
    set_bn_weights(conv_block.bn2, f"bn{stage}a_branch2b")
    set_bn_weights(conv_block.bn3, f"bn{stage}a_branch2c")
    
    set_conv_weights(conv_block.shortcut_conv, f"res{stage}a_branch1")
    set_bn_weights(conv_block.shortcut_bn, f"bn{stage}a_branch1")

#center
center_model = CenterInferenceModel()

set_conv_weights(center_model.conv1, 'conv1')
set_bn_weights(center_model.bn_conv1, 'bn_conv1')

set_conv_block_weights(center_model.conv_blocks[0], 2)
set_conv_block_weights(center_model.conv_blocks[1], 3)
set_conv_block_weights(center_model.conv_blocks[2], 4)

set_conv_weights(center_model.cen_hidden[0], 'conv2d_1')
set_conv_weights(center_model.cen_hidden[2], 'conv2d_2')

#size
size_model = SizeInferenceModel()

set_conv_weights(size_model.conv1, 'conv2d_3')
set_dense_weights(size_model.size_output[0], 'dense_1')









CenterInferenceModel(
  (conv1): Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2))
  (bn_conv1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_blocks): Sequential(
    (0): ConvBlock(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(2, 2))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
      (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut_conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2))
      (shortcut_bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inp

In [15]:
keras_input = np.ones(cfg.PREDICT_RESOLUTION)

torch_input = torch.rand([1]+list(reversed(cfg.PREDICT_RESOLUTION)))
inputs = torch.cat([torch_input, torch_input], dim=1)
print(inputs.shape)

print(inputs)


torch.Size([1, 6, 480, 480])
tensor([[[[0.1569, 0.5852, 0.5711,  ..., 0.2519, 0.5868, 0.6817],
          [0.2673, 0.3759, 0.0292,  ..., 0.6474, 0.6093, 0.2281],
          [0.3350, 0.8193, 0.1318,  ..., 0.5128, 0.5797, 0.2761],
          ...,
          [0.7428, 0.4713, 0.2815,  ..., 0.9936, 0.4039, 0.8545],
          [0.8331, 0.4620, 0.2620,  ..., 0.2550, 0.4954, 0.8870],
          [0.6030, 0.1082, 0.5090,  ..., 0.1117, 0.0558, 0.9911]],

         [[0.1609, 0.0820, 0.0150,  ..., 0.6952, 0.6103, 0.5648],
          [0.7785, 0.1722, 0.4269,  ..., 0.6344, 0.2961, 0.7239],
          [0.4731, 0.0936, 0.2933,  ..., 0.2071, 0.4467, 0.8108],
          ...,
          [0.7443, 0.3821, 0.8570,  ..., 0.7551, 0.7579, 0.6779],
          [0.1067, 0.8527, 0.7631,  ..., 0.2337, 0.3046, 0.9969],
          [0.4056, 0.7870, 0.8302,  ..., 0.8047, 0.7798, 0.5877]],

         [[0.0568, 0.8295, 0.1700,  ..., 0.2148, 0.9783, 0.0567],
          [0.7250, 0.6378, 0.4921,  ..., 0.4249, 0.0020, 0.8885],
          [0.

In [152]:

import cv2 

def get_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, cfg.PREDICT_RESOLUTION[:2])
    image = torch.from_numpy(image).float()
    image = image.permute(2, 0, 1).unsqueeze(0)
    return image
    

In [157]:
import ds_utils

K = 1
center_model.eval()
size_model.eval()

#torch_input1 = torch.rand([1]+list(reversed(cfg.PREDICT_RESOLUTION)))
#torch_input2 = torch.rand([1]+list(reversed(cfg.PREDICT_RESOLUTION)))

image_input = get_image("examples/1159362.jpg")
image_layout = torch.rand([1]+list(reversed(cfg.PREDICT_RESOLUTION)))

print(torch_input1.shape)
print(torch_input2.shape)
cen_feat, cen_probs = center_model(image_input, image_layout)

#print(cen_feat, cen_probs)

cen_feat, cen_probs = cen_feat.detach().numpy(), cen_probs.detach().numpy()

cen_probs = cen_probs.squeeze()
cen_inds = np.argsort(cen_probs)[::-1]
cen_inds = np.array(cen_inds[:K])

rois = ds_utils.centers_to_rois(cen_inds, [15,15], [15,15])

print(rois)

grids = np.zeros((0,2))
for i in range(K):
    input_roi  = rois[i].reshape((1, cfg.STATE_DIMS[0], cfg.STATE_DIMS[1]))
    size_probs = size_model(torch.from_numpy(cen_feat), torch.from_numpy(input_roi))
    size_probs = size_probs.detach().numpy()
    size_inds = np.argsort(size_probs.squeeze())[::-1]
    size_inds = np.array(size_inds[:K])
    for j in range(K):
        grids = np.vstack((grids, np.array([cen_inds[i], size_inds[j]]).reshape((1,2))))

xywhs = ds_utils.indices_to_boxes(grids, cfg.GRID_SHAPE)


torch.Size([1, 3, 480, 480])
torch.Size([1, 3, 480, 480])
[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]


In [158]:
print(grids)
print(xywhs)



[[148. 224.]]
[[0.9        0.6333333  0.96666664 0.96666664]]
